# Unzip the data

In [3]:
!unzip /content/Arabic_poetry_dataset.csv

Archive:  /content/Arabic_poetry_dataset.csv.zip
  inflating: Arabic_poetry_dataset.csv  
  inflating: __MACOSX/._Arabic_poetry_dataset.csv  


In [4]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [5]:
dataset = pd.read_csv('/content/Arabic_poetry_dataset.csv')
dataset.head(5)

,id,category,poet_name,poem_title,poem_text
0,0,الإمارات,خلفان بن مصبح,بدت تختال في حُلل الجمالِ,بدت تختال في حُلل الجمالِ\nوجادت بالزيارة والو...
1,1,الإمارات,خلفان بن مصبح,يا طائر الشعر القرير,يا طائر الشعر القرير\nيا وحي إلهام الصدور\nأسع...
2,2,الإمارات,خلفان بن مصبح,بنت حجرات أرى من عجب,بنت حجرات أرى من عجب\nأن أرى فيك جمال العرب\nد...
3,3,الإمارات,خلفان بن مصبح,هذا الربيع بنور الحسن وافانا,هذا الربيع بنور الحسن وافانا\nوقد كسا الأرض با...
4,4,الإمارات,خلفان بن مصبح,روحي فداك وإن مُنحتُ صدوداً,روحي فداك وإن مُنحتُ صدوداً\nأخفاك ما بي أم أط...


In [6]:
poems_count = 200
generated_poet = dataset[dataset['poet_name'] == 'جبران خليل جبران'].sample(frac=1)[:poems_count]
poems = np.array(generated_poet['poem_text'].values.tolist())

In [7]:
clean_poets = []
for i in poems:
    clean_poets.append(re.sub('([@A-Za-z0-9_ـــــــــــــ]+)|[^\w\s]|#|http\S+', ' ', i))

In [8]:
tokenizer = Tokenizer()
corpus = clean_poets

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
print(total_words)

5298


In [9]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [10]:
pad_seq = [len(x) for x in input_sequences]
max_sequence_len = max(pad_seq)
print(max_sequence_len)

496


In [12]:
model = tf.keras.models.load_model('jebran_poem_train.h5')

In [14]:
seed_text = "وكانت"
next_words = 10

for k in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)

وكانت من الله وعزة الخطب هونه غير المحمدات بعدك في القضاء
